# Use parameterize to turn a notebook into a widget.

![](https://user-images.githubusercontent.com/4236275/45729213-7c06a200-bb99-11e8-98f8-99cfaa81f16b.png)

In [16]:
Use `importnb.Parameterize` to create a default widget for a module.  We will display the widget in a `sidecar`.  

Use `importnb.Parameterize` to create a default widget for a module.  We will display the widget in a `sidecar`.


------> importnb.Parameterize
------> sidecar


<module 'sidecar' from '/Users/tonyfast/anaconda/envs/p6/lib/python3.6/site-packages/sidecar/__init__.py'>

In [18]:
import IPython, ipywidgets, sidecar, ast, pandas, importnb

Make a `pandas.DataFrame` if one doesn't exist.

In [20]:
df = IPython.get_ipython().user_ns.get('df', pandas.util.testing.makeDataFrame())

Our `Widget` `object` will take `importnb.Parameterize` expressions and expose them as `ipywidgets`.  The values may be changed and the module may be reloaded.

In [23]:
class Widget(importnb.Parameterize):
    def exec_module(self, module, sc = None):
        sc = sc or sidecar.Sidecar(title=self.name)
        with sc:
            reloader(module, sc)
            for name, value in module.__loader__.globals.items():
                object = ipywidgets.Text(description=name, value=str(value))
                IPython.display.display(object)
                observer(module, object)
            IPython.display.display(IPython.display.Markdown('---'))
            super().exec_module(module)

`observer` changes the `importnb.Parameterize.globals` so the module may be reloaded under different conditions.

In [31]:
def observer(module, widget):
    def caller(change):
        module.__loader__.globals.update({widget.description: ast.literal_eval(change['new'])})
    widget.observe(caller, names=['value'])

`reloader` creates a button to reload a module with.

In [33]:
def reloader(module, sc):
    def click(*_):
        with Widget(**module.__dict__):
            sc.clear_output()
            module.__loader__.exec_module(module, sc)
    button = ipywidgets.Button(description='reload')
    button.on_click(click)
    IPython.display.display(button)

`rows and columns` are some parameters to experiment with.

In [35]:
rows = 5
columns = ['A', 'B']

In [36]:
Creating the `_ipython_display_` function will create a default rich display view of the module.

Creating the `_ipython_display_` function will create a default rich display view of the module.


------> _ipython_display_


<function __main__._ipython_display_()>

In [37]:
def _ipython_display_():
    IPython.display.display(df.iloc[:rows][columns].describe())
    assert globals().get('nb').rows == rows, """The value was not updated"""

In [38]:
if __name__ == '__main__':
    with Widget(rows=rows, columns=columns):
        import __07_27_Untitled4 as nb

In [39]:
IPython.display.display(nb)

,A,B,C
count,10.000000,10.000000,10.000000
mean,0.337681,0.187146,-0.312534
std,1.116786,0.912780,1.059080
min,-1.307002,-1.391571,-2.285756
25%,-0.529809,-0.316863,-0.581636
50%,0.233798,0.118047,-0.440658
75%,1.236630,0.618056,0.184307
max,1.762426,2.038275,1.694824
